<a href="https://colab.research.google.com/github/abrichr/visual-contact-tracing/blob/master/Visual_Contact_Tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TODO

* Interpolate missing keypoints between frames
* Smooth keypoint positions across frames with optical flow: https://github.com/facebookresearch/DetectAndTrack/blob/d66734498a4331cd6fde87d8269499b8577a2842/lib/core/tracking_engine.py#L600
* Propagate keypoints through frames using optical flow and add to distance matrix: https://arxiv.org/pdf/1804.06208.pdf
* Add bounding box area difference to cost?
* Add legend to output video
* Hide subsequent hits
* Add padding in time around hits
* Hide numbers for non-infected tracks
* Re-number infected tracks
* Inter video tracking
* REST API (POST video/upload, GET video/contacts)
* improve highlight on padded hits (e.g. thick red border, animation)
* detect and blur faces

In [0]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1 cython_bbox
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
# install detectron2:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html
# get configs
# TODO: can we read this from site-packages?
!git clone https://github.com/facebookresearch/detectron2

In [0]:
# May need to restart your runtime prior to this to let installation take effect

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
#setup_logger()

import numpy as np
import cv2
import random
from google.colab import files
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog

from scipy.spatial.distance import pdist, squareform
import requests
#import subprocess as sp

import imageio
from cython_bbox import bbox_overlaps
from skimage.color import rgba2rgb

import os
import sys
import time
from tqdm.notebook import tqdm
from detectron2.utils import visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.video_visualizer import VideoVisualizer
import torch


Upload a video

In [0]:
if 0:
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn]))
    )

Or load from Google Drive

In [0]:

def download_file_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  with open(file_name, 'rb') as f:
    data = f.read()
    print('downloaded', len(data), 'bytes to', video_filename)


file_id = '0Bzf1l8WmTwu0eUluQ2h1NWZQRjQ'
video_filename = 'salsa_cpp_cam4.avi'
download_file_from_google_drive(file_id, video_filename)

Re-encode

In [0]:
RUN_CONFIGS = [
  # start_time_seconds, duration_seconds, start_infected_track
  (1, 10, 3),
  (10, 9.5, 2),
  (10, 14, 2),
  (1, 19.75, 3),
  (4, 5, 2)
]
run_config = RUN_CONFIGS[4]
start_time_seconds, duration_seconds, start_infected_track = run_config


ffmpeg = imageio.plugins.ffmpeg                                                
try:                                                                           
    ffmpeg.download()                                                          
except:                                                                        
    pass                                                                       
ffmpeg_exe = ffmpeg.get_exe()

video_path = video_filename
video_filename_reenc = video_filename + '-reenc.avi'

cmd_parts = [
  ffmpeg_exe,
  '-i', video_path,
  '-vcodec', 'h264',
  '-acodec', 'aac',
  #'-c', 'copy',
  '-strict',
  '-2',
  '-ss', str(start_time_seconds),
  '-t', str(duration_seconds),
  '-y',
  '-loglevel', 'debug',
  '-an',
  video_filename_reenc
]
cmd = ' '.join(cmd_parts)
print('Running cmd:\n', cmd)                                
! $cmd                                

with open(video_filename_reenc, 'rb') as f:
  data = f.read()
  print('wrote', len(data), 'bytes')

## Detectron Inference

In [0]:
SHOW_NUM_FRAMES = 0
WRITE_KP_OUTPUT = False
HIDE_KEYPOINTS = False
DEFAULT_CONFIG = 'detectron2/configs/COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml'
DEFAULT_CONF_THRESH = 0.7
DEFAULT_OPTS = [
  'MODEL.WEIGHTS',
  model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"),
]
# https://github.com/facebookresearch/detectron2/blob/04958b93e1232935e126c2fd9e6ccd3f57c3a8f3/detectron2/utils/visualizer.py#L32
KEYPOINT_THRESHOLD = 0.04
visualizer._KEYPOINT_THRESHOLD = KEYPOINT_THRESHOLD


def setup_cfg(config=DEFAULT_CONFIG, opts=DEFAULT_OPTS, conf_thresh=DEFAULT_CONF_THRESH):
  # load config from file and arguments
  cfg = get_cfg()


  # XXX untested
  if not torch.cuda.device_count():
    print('Running on CPU')
    cfg.MODEL.DEVICE = 'cpu'


  cfg.merge_from_file(config)
  cfg.merge_from_list(opts)
  # Set score_threshold for builtin models
  cfg.MODEL.RETINANET.SCORE_THRESH_TEST = conf_thresh
  cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = conf_thresh
  cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = conf_thresh
  cfg.freeze()
  return cfg


setup_logger(name='core')
logger = setup_logger('app')
logger.info(f'arguments: {[DEFAULT_CONFIG, DEFAULT_OPTS, DEFAULT_CONF_THRESH]}')

cfg = setup_cfg()
predictor = DefaultPredictor(cfg)

video_input = video_filename_reenc
print('video_input:', video_input)
assert os.path.isfile(video_input)
video = cv2.VideoCapture(video_input)
print('video:', video)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
print('frames_per_second:', frames_per_second)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('num_frames:', num_frames)
basename = os.path.basename(video_input)

kp_output_fname = f'{video_filename_reenc}-kp.mp4'
print('kp_output_fname:', kp_output_fname)
kp_output_file = cv2.VideoWriter(
  filename=kp_output_fname,
  fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
  fps=float(frames_per_second),
  frameSize=(width, height),
  isColor=True,
)

metadata = MetadataCatalog.get(
  cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused"
)
cpu_device = torch.device("cpu")
instance_mode = ColorMode.IMAGE
video_visualizer = VideoVisualizer(metadata, instance_mode)

def process_predictions(frame, predictions):
  frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
  if "panoptic_seg" in predictions:
    panoptic_seg, segments_info = predictions["panoptic_seg"]
    vis_frame = video_visualizer.draw_panoptic_seg_predictions(
      frame, panoptic_seg.to(cpu_device), segments_info
    )
  elif "instances" in predictions:
    predictions = predictions["instances"].to(cpu_device)
    if HIDE_KEYPOINTS:
      predictions.remove('pred_keypoints')
    vis_frame = video_visualizer.draw_instance_predictions(frame, predictions)
  elif "sem_seg" in predictions:
    vis_frame = video_visualizer.draw_sem_seg(
      frame, predictions["sem_seg"].argmax(dim=0).to(cpu_device)
    )

  # Converts Matplotlib RGB format to OpenCV BGR format
  vis_frame = cv2.cvtColor(vis_frame.get_image(), cv2.COLOR_RGB2BGR)
  return vis_frame

def _frame_from_video(video):
  while video.isOpened():
    success, frame = video.read()
    if success:
      yield frame
    else:
      break

frame_gen = _frame_from_video(video)
frames = []
all_predictions = []
for i, frame in enumerate(tqdm(frame_gen, total=num_frames)):

  frames.append(frame)
  start = time.time()
  predictions = predictor(frame)
  all_predictions.append(predictions)

  if i < SHOW_NUM_FRAMES or WRITE_KP_OUTPUT:
    vis_frame = process_predictions(frame, predictions)
    if i < SHOW_NUM_FRAMES:
      print('displaying frame', i)
      cv2_imshow(vis_frame)
    if WRITE_KP_OUTPUT:
      kp_output_file.write(vis_frame)

all_predictions = np.array(all_predictions)
#print('all_predictions.shape:', all_predictions.shape)

video.release()
kp_output_file.release()

if WRITE_KP_OUTPUT:
  with open(kp_output_fname, 'rb') as f:
    print('wrote', len(f.read()), 'bytes to', kp_output_fname)
  files.download(kp_output_fname)

## Create tracks

In [0]:
DEFAULT_HIDE_KEYPOINTS = True
DEFAULT_HIDE_BOXES = True
MAX_TRACK_IDS = 999
SHOW_START_END_FRAMES = 0
T = 60

# https://github.com/facebookresearch/detectron2/issues/754#issuecomment-579463185
JOINT_NAMES = [
  "nose",
  "left_eye", "right_eye",
  "left_ear", "right_ear",
  "left_shoulder", "right_shoulder",
  "left_elbow", "right_elbow",
  "left_wrist", "right_wrist",
  "left_hip", "right_hip",
  "left_knee", "right_knee",
  "left_ankle", "right_ankle"
]

# TODO: smoothing e.g. kalman or savgol
# https://stackoverflow.com/a/52450682/95989
max_num_instances = 0
all_keypoints = []
all_boxes = []
for predictions in all_predictions:
  #predictions['instances'].get_fields().keys()
  #'pred_boxes', 'scores', 'pred_classes', 'pred_keypoints'
  instances = predictions['instances'].to(cpu_device)
  #import pdb; pdb.set_trace()
  keypoints = np.asarray(instances.pred_keypoints)
  boxes = np.asarray(instances.pred_boxes.tensor)
  #print('keypoints:', keypoints.shape)
  all_keypoints.append(keypoints)
  all_boxes.append(boxes)
  num_instances = keypoints.shape[0]
  max_num_instances = max(max_num_instances, num_instances)
print('max_num_instances:', max_num_instances)
print('len(all_keypoints):', len(all_keypoints))


# https://github.com/facebookresearch/DetectAndTrack/blob/d66734498a4331cd6fde87d8269499b8577a2842/lib/core/tracking_engine.py#L106
def compute_pairwise_iou(a, b):
  """
  a, b (np.ndarray) of shape Nx4 and Mx4.
  The output is NxM, for each combination of boxes.
  """

  C = 1 - bbox_overlaps(
    np.ascontiguousarray(a, dtype=np.float64),
    np.ascontiguousarray(b, dtype=np.float64),
  )
  return C


def compute_distance_matrix(prev_boxes, cur_boxes):
  # TODO: consider keypoint distance?
  # TODO: weigh cost further away in time more heavily
  return compute_pairwise_iou(prev_boxes, cur_boxes)


# https://github.com/facebookresearch/DetectAndTrack/blob/d66734498a4331cd6fde87d8269499b8577a2842/lib/core/tracking_engine.py#L184
def bipartite_matching_greedy(C, prev_tracks):
    """
    Computes the bipartite matching between the rows and columns, given the
    cost matrix, C.
    """
    C = C.copy()  # to avoid affecting the original matrix
    prev_ids = []
    cur_ids = []
    MAX_COST = 1
    min_costs = []
    while (C == np.inf).sum() != C.size:
      #print('*' * 40)

      # Find the lowest cost element
      min_idx = C.argmin()
      i, j = np.unravel_index(min_idx, C.shape)
      min_val = C[i][j]
      #print('min_idx:', min_idx, 'min_val:', min_val, 'i:', i, 'j:', j)

      # Add to results
      min_cost = C.min()
      min_costs.append(min_cost)
      #print('adding to results:')
      prev_ids.append(i)
      if (not MAX_COST) or (min_cost < MAX_COST):
        cur_ids.append(j)
      else:
        cur_ids.append(-1)
      #print('prev_ids:', prev_ids)
      #print('cur_ids:', cur_ids)
      
      # Remove from cost matrix
      track = prev_tracks[i]
      #print('track:', track)
      track_idxs = [
        idx for idx in range(len(prev_tracks))
        if prev_tracks[idx] == track
      ]
      #print('track_idxs:', track_idxs)
      C[:, j] = np.inf
      for track_idx in track_idxs:
        #print('removing track_idx:', track_idx)
        C[track_idx, :] = np.inf
      #num_removed_costs = (C == np.inf).sum()
      #print('num_removed_costs:', num_infs)

    #mean_min_cost = np.mean(min_costs)
    #print('mean_min_cost:', mean_min_cost)

    #max_min_cost = np.max(min_costs)
    #print('max_min_cost:', max_min_cost)

    return prev_ids, cur_ids


def compute_matches(prev_boxes, cur_boxes, prev_tracks):
  assert len(prev_boxes) == len(prev_tracks)
  #matches = -np.ones((max_num_instances, ), dtype=np.int32)
  matches = -np.ones((len(cur_boxes), ), dtype=np.int32)
  if not prev_boxes.size:
    return matches
  C = compute_distance_matrix(prev_boxes, cur_boxes)
  prev_inds, next_inds = bipartite_matching_greedy(C, prev_tracks)
  #print('prev_inds:', prev_inds, len(prev_inds))
  #print('next_inds:', next_inds, len(next_inds))
  assert(len(prev_inds) == len(next_inds))
  for i in range(len(prev_inds)):
    #print('i:', i, 'next_inds[i]:', next_inds[i], 'prev_inds[i]', prev_inds[i])
    matches[next_inds[i]] = prev_inds[i]
    #print('matches:', matches)
  return matches

def get_frame_tracks(matches, prev_tracks, next_track_id):
  frame_tracks = []
  for i, m in enumerate(matches):
    #print('i:', i, 'm:', m, 'len(prev_tracks):', len(prev_tracks ))
    if m == -1 or m >= len(prev_tracks):  # didn't match to any
      frame_tracks.append(next_track_id[0])
      next_track_id[0] += 1
      if next_track_id[0] >= MAX_TRACK_IDS:
        # TODO: handle this
        print('Exceeded max track ids')
        next_track_id[0] %= MAX_TRACK_IDS
    else:
      frame_tracks.append(prev_tracks[m])
  #print('prev_tracks:\t', prev_tracks, 'len(prev_tracks):', len(prev_tracks))
  #print('frame_tracks:\t', frame_tracks, 'len(frame_tracks):', len(frame_tracks))
  return frame_tracks

def visualize_predictions(frame, instances, hide_keypoints=DEFAULT_HIDE_KEYPOINTS, hide_boxes=DEFAULT_HIDE_BOXES):
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    if hide_keypoints or hide_boxes:
      # make a copy
      instances = instances.to(cpu_device)
    if hide_keypoints:
      instances.remove('pred_keypoints')
    if hide_boxes:
      instances.pred_boxes.tensor = torch.Tensor()
    # https://github.com/facebookresearch/detectron2/blob/master/detectron2/utils/video_visualizer.py#L53
    # https://github.com/facebookresearch/detectron2/blob/b6fe828a2f3b2133f24cb93c1d0d74cb59c6a15d/detectron2/utils/video_visualizer.py#L53
    vis_frame = video_visualizer.draw_instance_predictions(frame, instances)
    # Converts Matplotlib RGB format to OpenCV BGR format
    vis_frame = cv2.cvtColor(vis_frame.get_image(), cv2.COLOR_RGB2BGR)
    return vis_frame

# compute tracks, inspired by:
# https://github.com/facebookresearch/DetectAndTrack/blob/d66734498a4331cd6fde87d8269499b8577a2842/lib/core/tracking_engine.py#L272
all_prev_boxes = []
video_tracks = []
next_track_id = [0]
for frame_id, (frame, predictions) in enumerate(tqdm(zip(frames, all_predictions), total=len(frames))):
  #print('\nframe_id:', frame_id)

  instances = predictions['instances'].to(cpu_device)  
  cur_boxes = np.asarray(instances.pred_boxes.tensor)
  prev_boxes = np.vstack(all_prev_boxes) if all_prev_boxes else np.array([])
  all_prev_tracks = video_tracks[
    max(0, frame_id - len(all_prev_boxes)) :
    max(0, frame_id)
  ]
  #print('len(all_prev_tracks):', len(all_prev_tracks))
  prev_tracks = np.hstack(all_prev_tracks) if all_prev_tracks else np.array([])
  #print('prev_boxes.shape:', prev_boxes.shape)
  #print('prev_tracks.shape:', prev_tracks.shape)

  matches = compute_matches(prev_boxes, cur_boxes, prev_tracks)
  #print('matches:\t', matches)
  # matches[i] contains the index of the box in the previous frames
  # corresponding to the box with index i in the current frame

  #print('prev_tracks:', prev_tracks)
  frame_tracks = get_frame_tracks(matches, prev_tracks, next_track_id)
  assert len(np.unique(frame_tracks)) == len(frame_tracks), (
      len(np.unique(frame_tracks)), len(frame_tracks)
  )
  video_tracks.append(np.array(frame_tracks))
  all_prev_boxes.append(cur_boxes)
  if len(all_prev_boxes) > T:
    all_prev_boxes = all_prev_boxes[1:]

  SHOW_FRAME_ON_NEW_TRACK = False
  HAS_NEW_MATCH = any([match == -1 for match in matches])
  if frame_id < SHOW_START_END_FRAMES or frame_id >= len(frames) - SHOW_START_END_FRAMES or (
      SHOW_FRAME_ON_NEW_TRACK and HAS_NEW_MATCH
  ):
    print('Visualizing frame_id:', frame_id)
    if HAS_NEW_MATCH:
      print('New match:')
    vis_frame = visualize_predictions(frame, instances)
    for box, frame_track in zip(cur_boxes, frame_tracks):
      cv2.putText(vis_frame, str(frame_track), (int(box[0]-5), int(box[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
      cv2.putText(vis_frame, str(frame_track), (int(box[0]-4), int(box[1]-4)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
    cv2_imshow(vis_frame)

# TODO: filter out large position jumps that immediately return after one frame

## Savgol Filter Keypoints over Time

In [0]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

DO_SAVGOL_FILTER = True
FILTER_WINDOW_LENGTH = 31
FILTER_POLY_ORDER = 2


num_tracks = 1 + max(max(frame_tracks) for frame_tracks in video_tracks)
print('num_tracks:', num_tracks)
target_joint_names = [
  'left_wrist',
  'right_wrist'
]
num_joints = len(target_joint_names)
print('num_joints:', num_joints)

# for plotting
vis_kp_positions_by_joint_track = {}

# for tracking
kp_positions_by_joint_track = {}
boxes_by_track = {}
for predictions, frame_tracks in zip(all_predictions, video_tracks):
  instances = predictions['instances'].to(cpu_device)
  keypoints = np.asarray(instances.pred_keypoints)
  boxes = np.asarray(instances.pred_boxes.tensor)
  for keypoints_per_instance, box, track in zip(keypoints, boxes, frame_tracks):
    for i_joint, joint_name in enumerate(target_joint_names):
      joint_idx = JOINT_NAMES.index(joint_name)
      joint_vals = keypoints_per_instance[joint_idx]
      joint_track = track * num_joints + i_joint
      x, y, prob = joint_vals
      kp_positions_by_joint_track.setdefault(joint_track, [])
      kp_positions_by_joint_track[joint_track].append((x, y))
    
    # for plotting only
    for joint_idx, joint_name in enumerate(JOINT_NAMES):
      # values in here don't necessarily correspond to similarly named values
      # elsewhere
      joint_vals = keypoints_per_instance[joint_idx]
      _joint_track = track * len(JOINT_NAMES) + joint_idx
      x, y, prob = joint_vals
      vis_kp_positions_by_joint_track.setdefault(_joint_track, [])
      vis_kp_positions_by_joint_track[_joint_track].append((x, y))

    boxes_by_track.setdefault(track, [])
    boxes_by_track[track].append(box)

  # fill in with nans for missing tracks
  all_tracks = set(range(num_tracks))
  missing_tracks = all_tracks - set(frame_tracks)
  #print('all_tracks:', all_tracks)
  #print('frame_tracks:', frame_tracks)
  #print('missing_tracks:', missing_tracks)
  for missing_track in missing_tracks:
    for i_joint, joint_name in enumerate(target_joint_names):
      joint_track = missing_track * num_joints + i_joint
      #print('joint_track:', joint_track)
      kp_positions_by_joint_track.setdefault(joint_track, [])
      kp_positions_by_joint_track[joint_track].append((np.nan, np.nan))
    for i_joint in range(len(JOINT_NAMES)):
      _joint_track = missing_track * len(JOINT_NAMES) + i_joint
      #print('_joint_track:', _joint_track)
      vis_kp_positions_by_joint_track.setdefault(_joint_track, [])
      vis_kp_positions_by_joint_track[_joint_track].append((np.nan, np.nan))
    #print('missing_track:', missing_track)
    boxes_by_track.setdefault(missing_track, [])
    boxes_by_track[missing_track].append([np.nan for _ in range(4)])

  # all tracks should now be of the same length
  for d in (kp_positions_by_joint_track, vis_kp_positions_by_joint_track, boxes_by_track):
    lengths = [len(positions) for positions in d.values()]
    #print('lengths:', len(lengths), lengths)
    assert np.all(np.array(lengths) == lengths[0]), lengths

filtered_boxes_by_track = {}
PLOT_BOX_FILTERING = True
for track, boxes in boxes_by_track.items():
  boxes = np.array(boxes)
  #print('boxes.shape:', boxes.shape)
  if DO_SAVGOL_FILTER:
    filtered_boxes = savgol_filter(
        boxes,
        FILTER_WINDOW_LENGTH,
        FILTER_POLY_ORDER,
        axis=0,
        mode='nearest'
    )
    nan_mask = np.isnan(filtered_boxes)
    filtered_boxes[nan_mask] = boxes[nan_mask]
  else:
    filtered_boxes = boxes
  
  # print('boxes:')
  # print(boxes)
  # print('filtered_boxes:')
  # print(filtered_boxes)
  assert len(filtered_boxes) == len(boxes), (
      len(filtered_boxes), len(boxes)
  )
  filtered_boxes_by_track[track] = filtered_boxes

  if PLOT_BOX_FILTERING:
    plt.figure()
    labels = ['x', 'y']
    plt.plot(boxes, label='raw', linestyle=':')
    plt.plot(filtered_boxes, label='filtered')
    plt.title(f'track {track} box')
    plt.xlabel('frame')
    plt.ylabel('position')
    plt.legend()
    plt.show()

# for visualization only
filtered_vis_kp_positions_by_joint_track = {}
for joint_track, vis_kp_positions in vis_kp_positions_by_joint_track.items():
  # TODO: resolve possible naming inconsistency
  track = joint_track // len(JOINT_NAMES)
  joint_idx = joint_track % len(JOINT_NAMES) 
  vis_kp_positions = np.array(vis_kp_positions)
  if DO_SAVGOL_FILTER:
    filtered_vis_kp_positions = savgol_filter(
        vis_kp_positions,
        FILTER_WINDOW_LENGTH,
        FILTER_POLY_ORDER,
        axis=0,
        mode='nearest'
    )
    nan_mask = np.isnan(filtered_vis_kp_positions)
    filtered_vis_kp_positions[nan_mask] = vis_kp_positions[nan_mask]
  else:
    filtered_vis_kp_positions = vis_kp_positions

  assert len(filtered_vis_kp_positions) == len(vis_kp_positions), (
      len(filtered_vis_kp_positions), len(vis_kp_positions)
  )
  filtered_vis_kp_positions_by_joint_track[joint_track] = filtered_vis_kp_positions

filtered_kp_positions_by_joint_track = {}
PLOT_KP_FILTERING = True
for joint_track, kp_positions in kp_positions_by_joint_track.items():
  # TODO: resolve possible naming inconsistency
  track = joint_track // num_joints
  joint_idx = joint_track % num_joints 

  joint_name = target_joint_names[joint_idx]
  kp_positions = np.array(kp_positions)
  if DO_SAVGOL_FILTER:
    filtered_kp_positions = savgol_filter(
        kp_positions,
        FILTER_WINDOW_LENGTH,
        FILTER_POLY_ORDER,
        axis=0,
        mode='nearest'
    )
    nan_mask = np.isnan(filtered_kp_positions)
    filtered_kp_positions[nan_mask] = kp_positions[nan_mask]
  else:
    filtered_kp_positions = kp_positions
  assert len(filtered_kp_positions) == len(kp_positions), (
      len(filtered_kp_positions), len(kp_positions)
  )
  filtered_kp_positions_by_joint_track[joint_track] = filtered_kp_positions

  if PLOT_KP_FILTERING:
    plt.figure()
    labels = ['x', 'y']
    plt.plot(kp_positions, label='raw', linestyle=':')
    plt.plot(filtered_kp_positions, label='filtered')
    plt.title(f'track {track} {joint_name}')
    plt.xlabel('frame')
    plt.ylabel('position')
    plt.legend()
    plt.show()

## Lowpass Filter Instances over Time

In [0]:
MIN_EMPTY_PADDING = 30
MAX_CONSECUTIVE_NONEMPTY_TO_REMOVE = 20

# for visualization
#for joint_track, filtered_vis_kp_positions in filtered_vis_kp_positions_by_joint_track.items():
#for track, filtered_boxes in filtered_boxes_by_track.items():

# for tracking
for joint_track, filtered_kp_positions in filtered_kp_positions_by_joint_track.items():
  N = len(filtered_kp_positions)
  for i in range(N):
    start_pad_start_idx = i - MIN_EMPTY_PADDING
    start_pad_end_idx = i - 1
    mid_start_idx = i
    mid_end_idx = i + MAX_CONSECUTIVE_NONEMPTY_TO_REMOVE - 1
    end_pad_start_idx = mid_end_idx + 1
    end_pad_end_idx = end_pad_start_idx + MIN_EMPTY_PADDING - 1
    if start_pad_start_idx < 0 or end_pad_end_idx >= N:
      continue
    #print('start_pad_start_idx:', start_pad_start_idx)
    #print('start_pad_end_idx:', start_pad_end_idx)
    #print('mid_start_idx:', mid_start_idx)
    #print('mid_end_idx:', mid_end_idx)
    #print('end_pad_start_idx:', end_pad_start_idx)
    #print('end_pad_end_idx:', end_pad_end_idx)

    start_pad_vals = filtered_kp_positions[start_pad_start_idx : start_pad_end_idx + 1]
    mid_vals = filtered_kp_positions[mid_start_idx : mid_end_idx + 1]
    end_pad_vals = filtered_kp_positions[end_pad_start_idx : end_pad_end_idx + 1]

    start_pad_mask = np.all(np.isnan(start_pad_vals), axis=1)
    #print('mid_vals.shape:', mid_vals.shape)
    mid_mask = np.all(np.isnan(mid_vals), axis=1)
    #print('mid_mask.shape:', mid_mask.shape)
    end_pad_mask = np.all(np.isnan(end_pad_vals), axis=1)

    num_start_pad_empty = start_pad_mask.sum()
    num_mid_nonempty = (~mid_mask).sum()
    num_end_pad_empty = end_pad_mask.sum()
    #print('i:', i, 'num_start_pad_empty:', num_start_pad_empty, 'num_mid_nonempty:', num_mid_nonempty, 'num_end_pad_empty:', num_end_pad_empty)
    if (
        num_start_pad_empty == MIN_EMPTY_PADDING and
        num_mid_nonempty > 0 and
        num_end_pad_empty == MIN_EMPTY_PADDING
    ):
      print('removing num_mid_nonempty:', num_mid_nonempty)
      filtered_kp_positions[mid_start_idx : mid_end_idx + 1] = np.nan

## Detect Infections

In [0]:
DISTANCE_THRESHOLD = 20
MIN_CONSECUTIVE_HIT_FRAMES = 1
USE_FILTERED_KEYPOINTS = DO_SAVGOL_FILTER and False
PLOT_FILTERED_KEYPOINTS = DO_SAVGOL_FILTER and True

print('DISTANCE_THRESHOLD:', DISTANCE_THRESHOLD)
print('MIN_CONSECUTIVE_HIT_FRAMES:', MIN_CONSECUTIVE_HIT_FRAMES)

print(len(all_predictions), 'frames')
infected_tracks = set(
  #[np.random.randint(0, max_num_instances)]
  [start_infected_track]
)
start_time = time.time()
print('len(all_keypoints):', len(all_keypoints))
print('len(video_tracks):', len(video_tracks))
assert(len(all_keypoints) == len(video_tracks))
dim = num_tracks * num_joints
consecutive_hits = np.zeros((dim, dim))
mean_keypoint_deltas = []
mean_box_deltas = []
frame_tups = []
print('Tracking infections...')
for frame_id, (keypoints, frame_tracks) in enumerate(
    tqdm(zip(all_keypoints, video_tracks), total=len(all_keypoints))
):
  #print('*' * 40)
  #print('frame_id:', frame_id)
  #print('infected_tracks:', infected_tracks)
  #print('frame_tracks:', frame_tracks)
  assert len(np.unique(frame_tracks)) == len(frame_tracks), (
      len(np.unique(frame_tracks)), len(frame_tracks)
  )

  target_joint_vals = []
  target_joint_probs = []

  for instance_idx, keypoints_per_instance in enumerate(keypoints):
    # https://github.com/facebookresearch/detectron2/blob/master/detectron2/utils/visualizer.py#L703
    # a tensor of shape (K, 3), where K is the number of keypoints
    # and the last dimension corresponds to (x, y, probability).

    for i_joint, joint_name in enumerate(target_joint_names):
      joint = JOINT_NAMES.index(joint_name)        
      joint_vals = keypoints_per_instance[joint]
      x, y, prob = joint_vals

      if USE_FILTERED_KEYPOINTS:
        track = frame_tracks[instance_idx]
        joint_track = track * num_joints + i_joint
        # TODO: resolve naming inconsistency: joint_vals <> kp_positions
        try:
          filtered_joint_vals = (
              filtered_kp_positions_by_joint_track[joint_track][frame_id]
          )
        except Exception as exc:
          # TODO: remove if no longer occuring, or understand why if it is
          print(exc)
          filtered_joint_vals = (
              filtered_kp_positions_by_joint_track[joint_track][-1]
          )
        #print('before x, y:', x, y)
        x, y = filtered_joint_vals 
        #print('after x, y:', x, y)

      # TODO: use confidence?
      #confident = prob >= _KEYPOINT_THRESHOLD

      if 0:
        print(
          'joint_name:', joint_name,
          'joint_vals:', joint_vals,
          'prob:', prob
        )
      target_joint_vals.append([x, y])
      target_joint_probs.append(prob)

  target_joint_vals = np.array(target_joint_vals)
  #print('target_joint_vals:', target_joint_vals.shape)

  distances = squareform(pdist(target_joint_vals))
  #print('distances.shape:', distances.shape)

  PRINT_DISTANCES = False
  if PRINT_DISTANCES:
    np.set_printoptions(
      threshold=sys.maxsize,
      formatter={'float': lambda x: "{0:0.0f}".format(x)}
    )
    print('distances:')
    print(distances)
    np.set_printoptions()
  
  hit_mask = distances < DISTANCE_THRESHOLD
  prob_mask = np.array(target_joint_probs) > KEYPOINT_THRESHOLD
  hit_mask[~prob_mask, :] = False
  hit_mask[:, ~prob_mask] = False
  #print('hit_mask.shape:', hit_mask.shape)
  frame_infected_tracks = set()
  #frame_tracks = np.array(frame_tracks)
  hit_count_by_joint_track_tup = {}
  joint_idx_by_joint_track = {}
  for infected_track in infected_tracks:
    #print('-' * 20)
    #print('infected_track:', infected_track)
    infected_idx = np.where(frame_tracks == infected_track)[0]
    if not infected_idx.size:
      #print('track', infected_track, 'no longer in frame')
      continue
    infected_idx = infected_idx[0]
    #print('infected_idx:', infected_idx)
    
    for i_joint in range(len(target_joint_names)):
      #print('. ' * 20)
      #print('i_joint:', i_joint)
      infected_joint_idx = infected_idx * len(target_joint_names) + i_joint
      row = hit_mask[infected_joint_idx]
      #print('row:', [1 if v else 0 for v in row ])
      #print('row.shape:', row.shape)
      hit_joint_idxs = np.where(row)[0].tolist()
      #print('hit_joint_idxs:', hit_joint_idxs)

      for hit_joint_idx in hit_joint_idxs:
        hit_track_idx = hit_joint_idx // num_joints
        hit_track = frame_tracks[hit_track_idx]

        infected_joint_track = infected_track * num_joints + i_joint
        hit_joint_track = hit_track * num_joints + i_joint

        if infected_joint_track == hit_joint_track:
          continue

        joint_idx_by_joint_track[infected_joint_track] = infected_joint_idx
        joint_idx_by_joint_track[hit_joint_track] = hit_joint_idx

        hit_count = 1 + (
            consecutive_hits[infected_joint_track][hit_joint_track]
        )
        joint_track_tup = (infected_joint_track, hit_joint_track)

        if joint_track_tup in hit_count_by_joint_track_tup:
          # This can happen if more than one of an infected person's joints
          # touch someone else's joint (e.g. double handshake).
          # Doing nothing treats this as a single hit.
          pass
        hit_count_by_joint_track_tup[joint_track_tup] = hit_count
  
  # TODO: don't fill if missing joint_idx belongs to track that's not visible
  # in this frame, or if it's within e.g. MIN_CONSECUTIVE_DEAD_KP_FRAMES frames
  # (it might reappear)
  consecutive_hits.fill(0)
  for (a_idx, b_idx), hit_count in hit_count_by_joint_track_tup.items():
    consecutive_hits[a_idx][b_idx] = hit_count
    consecutive_hits[b_idx][a_idx] = hit_count
  PRINT_CONSECUTIVE_HITS = False
  if PRINT_CONSECUTIVE_HITS:
    np.set_printoptions(
      threshold=sys.maxsize,
      formatter={'float': lambda x: "{0:0.0f}".format(x)}
    )
    print('consecutive_hits:')
    print(consecutive_hits)
    np.set_printoptions()

  hit_tracks = []
  all_hit_joint_track_tups = []
  consecutive_hit_joint_track_tups = []
  for joint_track_tup, hit_count in hit_count_by_joint_track_tup.items():
    infected_joint_track, hit_joint_track = joint_track_tup
    infected_track = infected_joint_track // num_joints
    hit_track = hit_joint_track // num_joints
    if infected_track == hit_track:
      continue

    all_hit_joint_track_tups.append(joint_track_tup)
    if hit_count < MIN_CONSECUTIVE_HIT_FRAMES:
      continue
    consecutive_hit_joint_track_tups.append(joint_track_tup)
    hit_track = hit_joint_track // num_joints
    hit_tracks.append(hit_track)

  #print('hit_tracks:', hit_tracks)
  frame_infected_tracks |= set(hit_tracks)

  frame = frames[frame_id]
  predictions = all_predictions[frame_id]
  instances = predictions['instances'].to(cpu_device)
  cur_boxes = np.asarray(instances.pred_boxes.tensor)

  if USE_FILTERED_KEYPOINTS or PLOT_FILTERED_KEYPOINTS:

    # TODO: rename
    pred_boxes = cur_boxes.copy()
    #print('pred_boxes:')
    #print(pred_boxes)
    num_instances = pred_boxes.shape[0]
    assert pred_boxes.shape == (num_instances, 4), (pred_boxes.shape, (num_instances, 4))
    pred_keypoints = instances.pred_keypoints.numpy().copy()
    #print('pred_keypoints:')
    #print(pred_keypoints)
    assert pred_keypoints.shape == (num_instances, len(JOINT_NAMES), 3), (pred_keypoints.shape, (num_instances, len(JOINT_NAMES), 3))
    box_deltas = []
    keypoint_deltas = []
    for instance_idx in range(num_instances):
      track = frame_tracks[instance_idx]
      filtered_boxes = filtered_boxes_by_track[track]

      track_box = filtered_boxes[frame_id]

      #print('before:', pred_boxes[instance_idx, :])
      box_delta = pred_boxes[instance_idx, :] - track_box
      box_deltas.append(box_delta)
      pred_boxes[instance_idx, :] = track_box
      #print('after:', pred_boxes[instance_idx, :])

      # filtered_vis_kp_positions_by_joint_track
      for i_joint in range(len(JOINT_NAMES)):
        joint_track = track * len(JOINT_NAMES) + i_joint
        filtered_vis_kp_positions = filtered_vis_kp_positions_by_joint_track[joint_track]
        track_positions = filtered_vis_kp_positions[frame_id]
        
        #print('before:', pred_keypoints[instance_idx][i_joint])
        keypoint_delta = pred_keypoints[instance_idx][i_joint][:2] - track_positions
        keypoint_deltas.append(keypoint_delta)
        pred_keypoints[instance_idx][i_joint][:2] = track_positions
        #print('after:', pred_keypoints[instance_idx][i_joint])

    mean_box_delta = np.abs(np.array(box_deltas)).mean()
    #print('mean_box_delta:', mean_box_delta)
    assert DO_SAVGOL_FILTER == bool(mean_box_delta), mean_box_delta
    mean_box_deltas.append(mean_box_delta)
    
    mean_keypoint_delta = np.abs(np.array(keypoint_deltas)).mean()
    #print('mean_keypoint_delta:', mean_keypoint_delta)
    assert DO_SAVGOL_FILTER == bool(mean_keypoint_delta), mean_keypoint_delta
    mean_keypoint_deltas.append(mean_keypoint_delta)

    before = instances.pred_boxes.tensor.numpy()
    instances.pred_boxes.tensor = torch.from_numpy(pred_boxes)
    after = instances.pred_boxes.tensor.numpy()
    assert DO_SAVGOL_FILTER == bool((before - after).mean())

    before = instances.pred_keypoints.numpy()
    instances.set('pred_keypoints', torch.from_numpy(pred_keypoints))
    after = instances.pred_keypoints.numpy()
    assert DO_SAVGOL_FILTER == bool((before - after).mean())

    cur_boxes = pred_boxes

  frame_tups.append((
    frame, instances, cur_boxes, frame_tracks, frame_infected_tracks,
    set(infected_tracks), consecutive_hit_joint_track_tups,
    all_hit_joint_track_tups, joint_idx_by_joint_track, target_joint_vals
  ))

  infected_tracks |= frame_infected_tracks

print('final infected_tracks:', infected_tracks)
duration = time.time() - start_time
print('duration:', duration)
mean_mean_box_delta = np.mean(mean_box_deltas)
print('mean_mean_box_delta:', mean_mean_box_delta)
mean_mean_keypoint_delta = np.mean(mean_keypoint_deltas)
print('mean_mean_keypoint_delta:', mean_mean_keypoint_delta)

'''
filter window length 61
mean_mean_box_delta: 1.9673217390508577
mean_mean_keypoint_delta: 2.335863792205027

filter window length 121
mean_mean_box_delta: 2.1252719775349673
mean_mean_keypoint_delta: 2.6383550710672163

filter window length 251
mean_mean_box_delta: 2.552904603603061
mean_mean_keypoint_delta: 2.911478190218577
'''
pass

## Pad hits

In [0]:
# XXX broken if PAD_HITS is False (see below)
PAD_HITS = False
HIGHLIGHT_AFTER_MIN_CONSECUTIVE_ONLY = False
PADDING_SECONDS = 1
# TODO: rename
r = 20
PADDING_FRAMES = int(PADDING_SECONDS * frames_per_second)
print('PADDING_FRAMES:', PADDING_FRAMES)

all_highlight_joint_track_tups = [set() for _ in range(len(frame_tups))]
all_boundaries_by_joint_track_tup = [dict() for _ in range(len(frame_tups))]

for frame_idx, (
    _,#frame,
    _,#instances,
    _,#cur_boxes,
    _,#frame_tracks,
    _,#frame_infected_tracks,
    _,#infected_tracks,
    consecutive_hit_joint_track_tups,
    all_hit_joint_track_tups,
    joint_idx_by_joint_track,
    target_joint_vals
) in tqdm(enumerate(frame_tups)):

  past_frame_idx = max(frame_idx - PADDING_FRAMES, 0)
  future_frame_idx = min(frame_idx + PADDING_FRAMES, len(frame_tups) - 1)
  if consecutive_hit_joint_track_tups:
    print('Padding hits at frame_idx:', frame_idx)
    # TODO: do this more efficiently
    for tup in consecutive_hit_joint_track_tups:
      if PAD_HITS:
        for _frame_idx in range(past_frame_idx, future_frame_idx + 1):
          print('_frame_idx:', _frame_idx, 'tup:', tup)

          infected_joint_track, hit_joint_track = tup
          infected_joint_idx = joint_idx_by_joint_track[infected_joint_track]
          hit_joint_idx = joint_idx_by_joint_track[hit_joint_track]


          all_highlight_joint_track_tups[_frame_idx].add(sorted_tup)  


          # XXX do the same thing if PAD_HITS is False


          infected_joint_vals = target_joint_vals[infected_joint_idx]
          hit_joint_vals = target_joint_vals[hit_joint_idx]
          xA, yA = infected_joint_vals
          xB, yB = hit_joint_vals
          cA = int(min(xA, xB)) - r
          cB = int(max(xA, xB)) + r
          rA = int(min(yA, yB)) - r
          rB = int(max(yA, yB)) + r

          # more than one pair of joints hits between the same pair of tracks
          all_boundaries_by_joint_track_tup[_frame_idx].setdefault(sorted_tup, [])
          all_boundaries_by_joint_track_tup[_frame_idx][sorted_tup].append((rA, rB, cA, cB))
      else:
        if HIGHLIGHT_AFTER_MIN_CONSECUTIVE_ONLY:
          highlight_joint_track_tups = consecutive_hit_joint_track_tups
        else:
          highlight_joint_track_tups = all_hit_joint_track_tups
        all_highlight_joint_track_tups[frame_idx] = highlight_joint_track_tups

In [0]:
WRITE_INF_OUTPUT = True
inf_output_fname = f'{video_filename_reenc}-inf.mp4'
print('inf_output_fname:', inf_output_fname)
inf_output_file = cv2.VideoWriter(
    filename=inf_output_fname,
    fourcc=cv2.VideoWriter_fourcc(*'mp4v'),
    fps=float(frames_per_second),
    frameSize=(width, height),
    isColor=True,
)


start_time = time.time()
print('Visualizing...')
for frame_id, ((
    frame,
    instances,
    cur_boxes,
    frame_tracks,
    frame_infected_tracks,
    infected_tracks,
    consecutive_hit_joint_track_tups,
    all_hit_joint_track_tups,
    joint_idx_by_joint_track,
    target_joint_vals
), highlight_joint_track_tups, boundaries_by_joint_track_tup) in tqdm(
    enumerate(zip(frame_tups, all_highlight_joint_track_tups, all_boundaries_by_joint_track_tup))
):
  print('*' * 40)
  print('frame_id:', frame_id)

  vis_frame = visualize_predictions(frame, instances, hide_keypoints=True, hide_boxes=True)
  for box, frame_track in zip(cur_boxes, frame_tracks):
    cv2.putText(vis_frame, str(frame_track), (int(box[0]-5), int(box[1]-5)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
    cv2.putText(vis_frame, str(frame_track), (int(box[0]-4), int(box[1]-4)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

  b_channel, g_channel, r_channel = cv2.split(vis_frame)
  alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 64

  # highlight infected
  #print('frame_infected_tracks:', frame_infected_tracks)
  #print('infected_tracks:', infected_tracks)
  highlight_tracks = frame_infected_tracks | infected_tracks
  #print('highlight_tracks:', highlight_tracks)
  highlight_boxes = []
  for track in highlight_tracks:
    track_id = np.where(frame_tracks == track)[0]
    if not track_id.size:
      # track is not visible in this frame
      continue
    track_id = track_id[0]
    #print('track_id:', track_id)
    box = cur_boxes[track_id]
    #print('box:', box)
    highlight_boxes.append(box)
  #print('highlight_boxes:', highlight_boxes)
  for box in highlight_boxes:
    #print('box:', box, 'box[0]:', box[0])
    xA = int(box[0])
    yA = int(box[1])
    xB = int(box[2])
    yB = int(box[3])
    alpha_channel[yA:yB, xA:xB] = 128
    alpha_channel[yA, xA:xB] = 255
    alpha_channel[yB, xA:xB] = 255
    alpha_channel[yA:yB, xA] = 255
    alpha_channel[yA:yB, xB] = 255

    for channel in b_channel, g_channel, r_channel:
      channel[yA, xA:xB] = 0
      channel[yB, xA:xB] = 0
      channel[yA:yB, xA] = 0
      channel[yA:yB, xB] = 0

  print('highlight_joint_track_tups:', highlight_joint_track_tups)
  # will not be true if pad_hits is false
  # assert set(highlight_joint_track_tups) == set(boundaries_by_joint_track_tup.keys()), (
  #     highlight_joint_track_tups, boundaries_by_joint_track_tup.keys()
  # )
  for joint_track_tup in highlight_joint_track_tups:
    infected_joint_track, hit_joint_track = joint_track_tup
    print('joint_track_tup:', joint_track_tup)

    infected_track = infected_joint_track // num_joints
    hit_track = hit_joint_track // num_joints
    sorted_tracks = sorted([infected_track, hit_track])
    sorted_joint_tracks = sorted([infected_joint_track, hit_joint_track])
    label = f'{sorted_tracks[0]}:{sorted_tracks[1]}'
    APPEND_JOINT_TRACKS_TO_LABEL = False
    if APPEND_JOINT_TRACKS_TO_LABEL:
      label +=  f';({sorted_joint_tracks[0]}:{sorted_joint_tracks[1]})'
    print('label:', label)
    
    def draw_boundary(boundary):
      print('draw_boundary() boundary:', boundary)
      rA, rB, cA, cB = boundary
      alpha_channel[rA:rB, cA:cB] = 255
      for channel in (b_channel, g_channel, r_channel):
        channel[rA, cA:cB] = 255
        channel[rB, cA:cB] = 255
        channel[rA:rB, cA] = 255
        channel[rA:rB, cB] = 255
        cv2.putText(channel, label, (cA-2, rA-2), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0))
        cv2.putText(channel, label, (cA-1, rA-1), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255))


    boundaries = boundaries_by_joint_track_tup.get(joint_track_tup)
    print('boundaries:', boundaries)
    if boundaries:
      for boundary in boundaries:
        draw_boundary(boundary)
    else:
      try:
        infected_joint_idx = joint_idx_by_joint_track[infected_joint_track]
        hit_joint_idx = joint_idx_by_joint_track[hit_joint_track]
      except KeyError as exc:
        print('exc:', exc)
        # A joint in the recent past or future that doesn't exist in the current
        # frame was involved in a hit
        continue

      infected_joint_vals = target_joint_vals[infected_joint_idx]
      hit_joint_vals = target_joint_vals[hit_joint_idx]
      xA, yA = infected_joint_vals
      xB, yB = hit_joint_vals
      cA = int(min(xA, xB)) - r
      cB = int(max(xA, xB)) + r
      rA = int(min(yA, yB)) - r
      rB = int(max(yA, yB)) + r
      boundary = (rA, rB, cA, cB)
      draw_boundary(boundary)
    
  img_BGRA = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))

  # TODO: move this to infection tracking cell
  new_infected_tracks = frame_infected_tracks - infected_tracks
  if new_infected_tracks:
    print('new_infected_tracks:', new_infected_tracks)
  
  # TODO: resolve naming inconsistency wrt word "all"
  # (elsewhere refers to frames, here refers to consecutive vs. not/all)
  if highlight_joint_track_tups or all_hit_joint_track_tups:
    print('frame_id:', frame_id, 'hit')
    cv2_imshow(img_BGRA)


  def bgra_2_bgr(bgra):
    # faster (?) but inverted alpha -> brightness
    b, g, r, a = cv2.split(bgra)
    bgr = cv2.merge((b, g, r))
    a = 255 - np.stack([a, a, a], axis=-1)
    bgr = cv2.addWeighted(np.float32(bgr) / 255, .5, np.float32(a) / 255, .5, 0)
    bgr = (bgr * 255).astype(np.uint8)
    '''
    # slower but matches cv2_imshow
    rgba = cv2.cvtColor(bgra, cv2.COLOR_BGRA2RGBA)
    rgb = rgba2rgb(rgba)
    rgb = (rgb * 255).astype(np.uint8)
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    '''
    return bgr

  if WRITE_INF_OUTPUT:
    img_BGR = bgra_2_bgr(img_BGRA)
    inf_output_file.write(img_BGR)

duration = time.time() - start_time
print('duration:', duration)

inf_output_file.release()

if WRITE_INF_OUTPUT:
  with open(inf_output_fname, 'rb') as f:
    print('wrote', len(f.read()), 'bytes to', inf_output_fname)
  ! ls -alh $inf_output_fname
  # this produces "MessageError: TypeError: Failed to fetch"
  #files.download(inf_output_fname)  

In [0]:
  if WRITE_INF_OUTPUT:
    ! ls -alh $inf_output_fname
    files.download(inf_output_fname)

Limitations / Future Work
- 2D results in false positives (due to lack of depth information) and false negatives (due to occlusion)
  - Lack of depth can be mitigated with depth estimation: https://roxanneluo.github.io/Consistent-Video-Depth-Estimation/
  - Occlusion can be mitigated by using mutiple cameras: https://arxiv.org/pdf/2003.03972v2.pdf
- Only keypoints are considered, not semantic segmentation masks, which may contain more information about whether contact ocurred
- Doesn't track individuals between videos

In [0]:
print('frames_per_second:', frames_per_second)
print('num_frames:', num_frames)